# This notebook runs the Deep Gravity model on the yearly trade prediction dataset

In [1]:
import pandas as pd
import numpy as np
import datetime
import sys
import tqdm

import random
import torch.optim as optim
import torch.utils.data.distributed
from torch import nn

import model_utils
from data_compiler import FlowDataset
from deepgravity import DeepGravity

sys.path.append('../trade_predictions')
import parameters

# random seeds
torch.manual_seed(parameters.seed)
np.random.seed(parameters.seed)
random.seed(parameters.seed)

torch_device = torch.device("cpu")


/Users/rgyuri/.pyenv/versions/3.10.2/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
##############
# Load data
##############

nodes = pd.read_csv(parameters.node_path)
nodes_columns = parameters.node_features + [parameters.node_id] + [parameters.node_timestamp]
nodes = nodes[nodes_columns]

edges = pd.read_csv(parameters.edge_path)
edges_columns = parameters.flows_features + [parameters.flow_origin] + \
    [parameters.flow_destination] + [parameters.flows_timestamp] + \
        [parameters.flows_value]
edges = edges[edges_columns]

##############
# Initial cleaning
##############

nodes = nodes.fillna(0)

##############
# Create data objects
##############

columns = {'node_id': parameters.node_id,
           'node_timestamp': parameters.node_timestamp,
           'flow_origin': parameters.flow_origin,
           'flow_destination': parameters.flow_destination,
           'flows_timestamp': parameters.flows_timestamp,
           'flows_value':parameters.flows_value}

flow_data = FlowDataset(columns=columns,
                        unit = [parameters.flow_origin, parameters.flows_timestamp],
                        nodes=nodes,
                        edges=edges,)

# Create a list of FlowDataset objects
flow_data_chunked = flow_data.create_chunks(chunk_size=6)

# Add past values to each chunk
[flow_chunk.add_past_values(periods=parameters.lag_periods,
                            edge_columns = parameters.time_dependent_edge_columns,
                            node_columns = parameters.time_dependent_node_columns) for flow_chunk in tqdm.tqdm(flow_data_chunked)]

# Add target to each chunk
[flow_chunk.add_target_values() for flow_chunk in tqdm.tqdm(flow_data_chunked)]

# Create a list of FlowDataset objects
train_data_chunked = []
test_data_chunked = []

for flow_data in tqdm.tqdm(flow_data_chunked):
    train_data, test_data = flow_data.split_train_test(test_period = 1)
    train_data_chunked.append(train_data)
    test_data_chunked.append(test_data)


100%|██████████| 20/20 [00:00<00:00, 392.14it/s]


In [3]:
prediction_list = []
for chunk in range(len(train_data_chunked)):
    train_data_loader = torch.utils.data.DataLoader(train_data_chunked[chunk], batch_size=parameters.batch_size)
    test_data_loader = torch.utils.data.DataLoader(test_data_chunked[chunk], batch_size=parameters.batch_size)

    input_dim = train_data_chunked[chunk].get_feature_dim()

    deep_gravity_model = DeepGravity(dim_input = input_dim,
                                    dim_hidden = parameters.dim_hidden)
    
    optimizer = optim.RMSprop(deep_gravity_model.parameters(), lr=parameters.lr, momentum=parameters.momentum)

    for t in range(parameters.epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        model_utils.train(train_data_loader, deep_gravity_model, optimizer)
        model_utils.test(test_data_loader, deep_gravity_model, test_data_chunked[chunk], loss_fn = None)

    model_utils.test(test_data_loader, deep_gravity_model, test_data_chunked[chunk], loss_fn = None, store_predictions=True)
    prediction_list.append(test_data_chunked[chunk].compile_predictions(columns_to_rename = parameters.columns_to_rename))
    print("Done!")
pd.concat(prediction_list, axis=0).to_csv(f"{parameters.output_path}/prediction_{str(datetime.datetime.now()).replace(' ', '_')[:19]}.csv")

Epoch 1
-------------------------------
loss: 249872096.000000  [   10/  258]
loss: 166969216.000000  [  110/  258]
loss: 68005112.000000  [  210/  258]
Test Error: Avg loss: 2416031.333333 

Epoch 2
-------------------------------
loss: 42864516.000000  [   10/  258]
loss: 63920668.000000  [  110/  258]
loss: 72086968.000000  [  210/  258]
Test Error: Avg loss: 1652146.138889 

Epoch 3
-------------------------------
loss: 134415008.000000  [   10/  258]
loss: 93359896.000000  [  110/  258]
loss: 34570000.000000  [  210/  258]
Test Error: Avg loss: 1168637.625000 

Epoch 4
-------------------------------
loss: 60600500.000000  [   10/  258]
loss: 46320940.000000  [  110/  258]
loss: 159644672.000000  [  210/  258]
Test Error: Avg loss: 812213.875000 

Epoch 5
-------------------------------
loss: 39048920.000000  [   10/  258]
loss: 55640260.000000  [  110/  258]
loss: 84760536.000000  [  210/  258]
Test Error: Avg loss: 677874.975694 

Epoch 6
-------------------------------
loss: 62

In [ ]:
#set([i[1] for i in train_data_chunked[0].data_dict.keys()])
#train_data_chunked[0].data_dict[(32, 1996)].columns
#data_loader = torch.utils.data.DataLoader(flow_data_chunked[0], batch_size=parameters.batch_size)
#for X, y in data_loader:
#    print(f"Shape of X : {X.shape}")
#    print(f"Shape of y: {y.shape} {y.dtype}")
#    break